# Getting started with the EB-NeRD

In [1]:
from pathlib import Path
import polars as pl

from recsys_challenge.dataset._vocab import WordVocab

from recsys_challenge.dataset.preprocess.examples import (load_hop_dict, build_examples)

### Load dataset

In [2]:
PATH = Path("../data/small")
VOCAB_PATH = Path("../data/vocab")
NEIGHBOR_PATH = Path("../data/neighbors")
SPLIT = "validation"

OUTPUT_PATH = Path("../data/examples")

In [3]:
df_behaviors = pl.read_parquet(PATH / SPLIT / "behaviors.parquet")

### Load vocab

In [4]:
vocab_articles = WordVocab.load_vocab(VOCAB_PATH / "articles_id_vocab.bin")
vocab_user = WordVocab.load_vocab(VOCAB_PATH / "user_id_vocab.bin")

### Load neighbors

In [5]:
user_one_hop = load_hop_dict(NEIGHBOR_PATH / f"{SPLIT}-user_one_hops.txt")
print([k for k in user_one_hop.keys()][:-10])
article_one_hop = load_hop_dict(NEIGHBOR_PATH / f"{SPLIT}-article_one_hops.txt")
print([k for k in article_one_hop.keys()][:-10])

user_two_hop = load_hop_dict(NEIGHBOR_PATH / f"{SPLIT}-user_two_hops.txt")
print([k for k in user_two_hop.keys()][:-10])
article_two_hop = load_hop_dict(NEIGHBOR_PATH / f"{SPLIT}-article_two_hops.txt")
print([k for k in article_two_hop.keys()][:-10])


../data/neighbors/validation-user_one_hops.txt error lines: 0
[6592, 862, 3818, 5608, 129, 796, 2656, 543, 719, 11205, 5616, 7186, 10287, 3831, 1578, 4056, 12172, 1579, 3627, 873, 2797, 5968, 3260, 5066, 3855, 2261, 1654, 2525, 2262, 5069, 8908, 1586, 6794, 2136, 223, 7909, 2712, 1914, 1492, 964, 2646, 4479, 4760, 7677, 9498, 1842, 4537, 86, 1666, 1594, 4335, 105, 9019, 185, 12595, 436, 4585, 3689, 6058, 3502, 605, 9075, 773, 7381, 817, 10666, 3515, 3319, 3516, 6919, 10669, 189, 502, 107, 4405, 6907, 5144, 12675, 3151, 4606, 6916, 9835, 1034, 1332, 4686, 3776, 1629, 11009, 10101, 5136, 214, 1601, 1870, 2173, 1960, 6498, 5154, 11631, 2303, 6503, 1961, 1217, 2068, 81, 14080, 3937, 10685, 1872, 9870, 6105, 9875, 12733, 1742, 7736, 1834, 518, 36, 5982, 2393, 6799, 2811, 4797, 1068, 414, 4205, 252, 2477, 2214, 3864, 1929, 486, 2133, 884, 5664, 1308, 3115, 390, 2409, 2150, 356, 524, 2155, 1320, 1449, 1760, 1761, 2697, 1321, 9050, 9752, 3906, 1856, 3496, 6854, 4346, 2419, 3182, 200, 2878, 318

In [6]:
df_behaviors.head()

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
u32,i32,datetime[μs],f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,f32,f32
96791,null,2023-05-28 04:21:24,9.0,null,2,"[9783865, 9784591, … 9784710]",[9784696],22548,false,null,null,null,false,142,72.0,100.0
96798,null,2023-05-28 04:31:48,46.0,null,2,"[9782884, 9783865, … 9784648]",[9784281],22548,false,null,null,null,false,143,16.0,28.0
96801,null,2023-05-28 04:30:17,14.0,null,2,"[9784648, 7184889, … 9781983]",[9784444],22548,false,null,null,null,false,143,12.0,24.0
96808,null,2023-05-28 04:27:19,22.0,null,2,"[9784607, 9695098, … 9781983]",[9781983],22548,false,null,null,null,false,142,125.0,80.0
96810,null,2023-05-28 04:29:47,23.0,null,2,"[9781983, 7184889, … 9781520]",[9784642],22548,false,null,null,null,false,142,null,null


In [7]:
match SPLIT:
    case "train":
        output_name = "training_examples.tsv"
    case "validation":
        output_name = "validation_examples.tsv"
    case "test":
        output_name = "test_examples.tsv"
    case _:
        raise ValueError(f"Unknown split {SPLIT}")


build_examples(
    df_behaviors,
    vocab_user,
    vocab_articles,
    user_one_hop,
    article_one_hop,
    user_two_hop,
    article_two_hop,
    OUTPUT_PATH,
    output_name=output_name,
)

Building examples:   0%|          | 0/244647 [00:00<?, ?it/s]